In [1]:
import sys
import numpy as np

from sklearn.metrics import mean_squared_error

sys.path.append('..')
from higgs_inference.various.utils import format_number

In [2]:
result_dir = '../results/'

# TablePrinter class

In [3]:
class TablePrinter:
    
    def __init__(self, metric_fns=[], header=None, precisions=[]):
        
        # Functions for metrics
        self.metric_fns = metric_fns
        self.n_metrics = len(self.metric_fns)
        self.precisions = precisions if len(precisions) == self.n_metrics else [2] * self.n_metrics
        
        # Total table and current block
        self.table = ''
        self.block_entries = []
        self.block_brackets = []

        # Formatting options
        self.indent = '   '
        self.col_sep = ' & '
        self.end_row = r'\\'
        self.midrule = r'\midrule'
        self.end_line = '\n'
        self.emphasis_begin = r'\mathbf{'
        self.emphasis_end = r'}'
        
        # Header
        self.table = ''
        if header is not None:
            self.table += self.indent + header + self.end_row + self.end_line
    
    
    def finalise_block(self):

        # Find best performance
        block_metrics = [line[2:] for line in self.block_entries]
        block_metrics = np.array(block_metrics)
        block_best = []
        for i in range(self.n_metrics):
            try:
                block_best.append(np.nanargmin(block_metrics[:,i]))
            except ValueError:
                block_best.append(-1)

        # Format entries
        text = ''
        for i, (line, brackets) in enumerate(zip(self.block_entries, self.block_brackets)):
            
            # Labels
            text += self.indent + line[0] + self.col_sep + line[1] + self.col_sep
            
            # Metrics
            for j in range(self.n_metrics):
                if np.isfinite(line[j + 2]):
                    if brackets[j+2]:
                        text += '(' + format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j])) + ')'
                    else:
                        text += format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j]))
                if j == len(line) - 3:
                    text += self.end_row + self.end_line
                else:
                    text += self.col_sep

        # Add to document and reset for next block
        self.table += text
        self.block_entries = []
        self.block_brackets = []
    
    
    def new_block(self):
        self.finalise_block()
        self.table += self.indent + self.midrule + self.end_line
    
    
    def add(self, col1, col2, filename, folder='parameterized'):
        
        # Label columns
        line = [col1, col2]
        brackets = [False, False]
        
        # Metrics
        for fn in self.metric_fns:
            bracket = False
            try:
                value = fn(filename, folder)
            except (IOError, ValueError):
                #print('File', filename, 'in folder', folder, 'not found')
                value = np.nan
                
            if isinstance(value, (list, tuple)):
                value, bracket = value
                
            line.append(value)
            brackets.append(bracket)

        self.block_entries.append(line)
        self.block_brackets.append(brackets)
    
    
    def print(self):
        self.finalise_block()
        return self.table

# Metrics

In [4]:
def mse_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return mean_squared_error(log_r_truth, log_r_estimated)
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return mean_squared_error(log_r_truth[finites], log_r_estimated[finites]), True

def mse_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_trained_' + filename + '.npy'))
    try:
        return mean_squared_error(log_r_truth, log_r_estimated)
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return mean_squared_error(log_r_truth[finites], log_r_estimated[finites]), True

def mse_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    try:
        return mean_squared_error(expected_log_r_truth, expected_log_r_estimated)
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites]), True

def mse_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    try:
        return mean_squared_error(expected_log_r_truth, expected_log_r_estimated)
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites]), True

def mse_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    return mean_squared_error(t_truth, t_estimated)

def mse_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    return mean_squared_error(t_truth, t_estimated)

def var_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    return np.var(expected_log_r_truth - expected_log_r_estimated)

def var_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    return np.var(expected_log_r_truth - expected_log_r_estimated)

# Main tables

In [11]:
labels = [r'AFC (density est.\ in $\boldx$)',
          'carl (PbP, raw)',
          'carl (PbP, cal.)',
          'carl (param., raw)',
          'carl (param., cal.)',
          'carl + score (param., raw)',
          'carl + score (param., cal.)',
          'SM score regression',
          'Ratio regression (PbP, raw)',
          'Ratio regression (param., raw)',
          r'Ratio regr.\ + score (param., raw)']

folders = (['afc']
          + ['point_by_point'] * 2 + ['parameterized'] * 4
          + ['score_regression']
          + ['point_by_point'] + ['parameterized'] * 2)

filenames = ['afc',
             'carl',
             'carl_calibrated',
             'carl',
             'carl_calibrated',
             'combined',
             'combined_calibrated',
             'scoreregression',
             'regression',
             'regression',
             'combinedregression']


def show_main_table(algorithm_begin=0, algorithm_end=None):
    
    table = TablePrinter([mse_log_r_trained, mse_log_r_nottrained,
                      mse_expected_log_r, mse_delta_expected_log_r],
                      precisions=[3,3,1,1])

    for i, (label, filename, folder) in enumerate(
        zip(labels[algorithm_begin:algorithm_end], filenames[algorithm_begin:algorithm_end], folders[algorithm_begin:algorithm_end])):

        if i > 0:
            table.new_block()

        if folder == 'point_by_point':
            #table.add(label, 'PbP (1)', filename + '_shallow', 'point_by_point')
            table.add(label, 'PbP (2)', filename, 'point_by_point')
            #table.add('', 'PbP (3)', filename + '_deep', 'point_by_point')

        elif folder == 'afc':
            #table.add(label, '2d (1)', filename + '_2d_shallow', 'afc')
            table.add(label, '2d (2)', filename + '_2d', 'afc')
            #table.add('', '2d (3)', filename + '_2d_deep', 'afc')

            #table.add(label, '5d (1)', filename + '_5d_shallow', 'afc')
            table.add(label, '5d (2)', filename + '_5d', 'afc')
            #table.add('', '5d (3)', filename + '_5d_deep', 'afc')

        elif folder == 'score_regression':
            #table.add('', r'Fixed density est. on $t$ (1)', filename + '_score_shallow', 'score_regression')
            table.add(label, r'Fixed density est. on $t$  (2)', filename + '_score', 'score_regression')
            #table.add('', r'Fixed density est. on $t$  (3)', filename + '_score_deep', 'score_regression')

            #table.add('', r'Dynamic density est. on $t$  (1)', filename + '_rotatedscore_shallow', 'score_regression')
            table.add('', r'Dyn. dens. est. on $t$  (2)', filename + '_rotatedscore', 'score_regression')
            #table.add('', r'Dynamic density est. on $t$  (3)', filename + '_rotatedscore_deep', 'score_regression')

            #table.add('', r'Density est. on $t \cdot \theta$  (1)', filename + '_scoretheta_shallow', 'score_regression')
            table.add('', r'Dens. est. on $t \cdot \theta$  (2)', filename + '_scoretheta', 'score_regression')
            #table.add('', r'Density est. on $t \cdot \theta$  (3)', filename + '_scoretheta_deep', 'score_regression')

        else:
            #table.add(label, 'Baseline (1)', filename + '_shallow')
            table.add(label, 'Baseline (2)', filename)
            #table.add('', 'Baseline (3)', filename + '_deep')

            #table.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
            table.add('', r'Random $\boldtheta$ (2)', filename + '_random')
            #table.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

            #table.add('', 'Physics-aware, baseline (1)', filename + '_aware_shallow')
            table.add('', 'Aware, baseline (2)', filename + '_aware')
            #table.add('', 'Physics-aware, baseline (3)', filename + '_aware_deep')

            #table.add('', 'Physics-aware, morphing basis (1)', filename + '_aware_basis_shallow')
            table.add('', 'Aware, basis (2)', filename + '_aware_basis')
            #table.add('', 'Physics-aware, morphing basis (3)', filename + '_aware_basis_deep')

    print(table.print())

In [12]:
show_main_table(0,5)

   AFC (density est.\ in $\boldx$) & 2d (2) & \emph{0.147} & \emph{0.209} & \emph{27.2} & 27.8\\
   AFC (density est.\ in $\boldx$) & 5d (2) & 0.176 & 0.323 & 32.9 & \emph{20.9}\\
   \midrule
   carl (PbP, raw) & PbP (2) & \emph{0.091} & \emph{0.147} & \emph{369.4} & \emph{269.6}\\
   \midrule
   carl (PbP, cal.) & PbP (2) & \emph{0.047} & \emph{0.072} & \emph{12.3} & \emph{8.3}\\
   \midrule
   carl (param., raw) & Baseline (2) & \emph{0.061} & \emph{0.123} & \emph{13.7} & \emph{17.5}\\
    & Random $\boldtheta$ (2) & 0.074 & 0.131 & 69.6 & 26.7\\
    & Aware, baseline (2) &  &  & (93.8) & (83.3)\\
    & Aware, basis (2) &  &  &  & \\
   \midrule
   carl (param., cal.) & Baseline (2) & \emph{0.055} & \emph{0.104} & 6.8 & 6.0\\
    & Random $\boldtheta$ (2) & 0.057 & 0.122 & \emph{2.8} & \emph{2.8}\\
    & Aware, baseline (2) &  &  &  & \\
    & Aware, basis (2) &  &  &  & \\



/Users/johannbrehmer/anaconda3/envs/higgs_inference/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


In [13]:
show_main_table(5,8)

   carl + score (param., raw) & Baseline (2) & 0.123 & \emph{0.137} & 572.6 & \emph{1.1}\\
    & Random $\boldtheta$ (2) & \emph{0.101} & 0.190 & 130.7 & 42.5\\
    & Aware, baseline (2) &  &  & (\emph{69.0}) & (58.3)\\
    & Aware, basis (2) &  &  &  & \\
   \midrule
   carl + score (param., cal.) & Baseline (2) & \emph{0.019} & \emph{0.032} & \emph{3.6} & \emph{2.5}\\
    & Random $\boldtheta$ (2) & 0.065 & 0.145 & 20.9 & 18.8\\
    & Aware, baseline (2) &  &  &  & \\
    & Aware, basis (2) &  &  &  & \\
   \midrule
   SM score regression & Fixed density est. on $t$  (2) & 0.038 & 0.037 & \emph{2.4} & \emph{1.2}\\
    & Dyn. dens. est. on $t$  (2) & \emph{0.036} & 0.035 & 2.7 & 1.4\\
    & Dens. est. on $t \cdot \theta$  (2) & 0.052 & \emph{0.034} & 7.9 & 5.3\\



In [14]:
show_main_table(8,None)

   Ratio regression (PbP, raw) & PbP (2) & \emph{0.004} & \emph{0.009} & \emph{10.8} & \emph{12.1}\\
   \midrule
   Ratio regression (param., raw) & Baseline (2) & \emph{0.009} & 0.021 & 14.3 & 13.3\\
    & Random $\boldtheta$ (2) & 0.010 & \emph{0.018} & \emph{0.5} & \emph{0.6}\\
    & Aware, baseline (2) & 0.277 & 0.457 & 32.4 & 53.2\\
    & Aware, basis (2) & 0.108 & 69.365 & 73428.2 & 141763.6\\
   \midrule
   Ratio regr.\ + score (param., raw) & Baseline (2) & \emph{0.008} & \emph{0.016} & 5.6 & 3.5\\
    & Random $\boldtheta$ (2) & 0.013 & 0.027 & \emph{1.2} & \emph{1.2}\\
    & Aware, baseline (2) &  &  & 195.2 & 194.7\\
    & Aware, basis (2) &  &  &  & \\



# Score accuracy table

In [10]:
labels = ['carl',
          #'Score',
          'carl + score',
          'Ratio regression',
          'Ratio regr.\ + score']

filenames = ['carl',
             #'score',
             'combined',
             'regression',
             'combinedregression']

table4 = TablePrinter([mse_score_trained, mse_score_nottrained],
                      precisions=[2,2])

for i, (label, filename) in enumerate(zip(labels, filenames)):
    
    if i > 0:
        table4.new_block()
        
    #table4.add(label, 'Baseline (1)', filename + '_shallow')
    table4.add(label, 'Baseline (2)', filename)
    #table4.add('', 'Baseline (3)', filename + '_deep')
    
    #table4.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
    table4.add('', r'Random $\boldtheta$ (2)', filename + '_random')
    #table4.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

    #table4.add('', 'Physics-aware, baseline (1)', filename + '_aware_shallow')
    table4.add('', 'Aware, baseline (2)', filename + '_aware')
    #table4.add('', 'Physics-aware, baseline (3)', filename + '_aware_deep')

    #table4.add('', 'Physics-aware, morphing basis (1)', filename + '_aware_basis_shallow')
    table4.add('', 'Aware, basis (2)', filename + '_aware_basis')
    #table4.add('', 'Physics-aware, morphing basis (3)', filename + '_aware_basis_deep')

print(table4.print())

   carl & Baseline (2) & \emph{3.09} & 759.39\\
    & Random $\boldtheta$ (2) & 3.42 & \emph{758.92}\\
    & Aware, baseline (2) &  & \\
    & Aware, basis (2) &  & \\
   \midrule
   carl + score & Baseline (2) & \emph{1.38} & \emph{758.98}\\
    & Random $\boldtheta$ (2) & 2.44 & 759.26\\
    & Aware, baseline (2) &  & \\
    & Aware, basis (2) &  & \\
   \midrule
   Ratio regression & Baseline (2) & \emph{1.47} & 758.01\\
    & Random $\boldtheta$ (2) & 1.53 & \emph{757.58}\\
    & Aware, baseline (2) & 509.49 & 857.51\\
    & Aware, basis (2) & 1008789.83 & 1053.95\\
   \midrule
   Ratio regr.\ + score & Baseline (2) & \emph{1.34} & \emph{754.67}\\
    & Random $\boldtheta$ (2) & 1.60 & 756.35\\
    & Aware, baseline (2) &  & \\
    & Aware, basis (2) &  & \\

